###  imports & display precision
- Imports NumPy, masked arrays (`ma`), Pandas.
- Brings in TensorFlow/Keras for neural nets.
- Imports scalers and train/test split from scikit-learn.
- `tabulate` for HTML tables when showing nearest neighbors.
- Loads helper functions from `recsysNN_utils` (provided by the assignment).
- Sets Pandas display precision.

In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)

###load sample content-based tables
- Reads precomputed content-based recommendation examples.
- Displays the “top 10” table.

In [ ]:
top10_df = pd.read_csv("./data/content_top10_df.csv")
bygenre_df = pd.read_csv("./data/content_bygenre_df.csv")
top10_df

### show second content table
- Displays per-genre content recommendations (for context).

In [ ]:
bygenre_df

### load training data & set layout indices
- `load_data()` returns user/item feature matrices, targets, and metadata.
- Computes feature counts used by the NN (excludes IDs and selected non-train columns).
- Records column offsets for user/item slices used during training.

In [ ]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, item

###  peek at user training rows
- Pretty-prints a few user rows with selected columns for inspection.

In [ ]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

### Cell 6 — peek at item training rows
- Pretty-prints a few item rows with selected columns for inspection.

In [ ]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

###  look at first few targets
- Prints first five target ratings (training labels).

In [ ]:
print(f"y_train[:5]: {y_train[:5]}")

###  scale features & target
- Keeps unscaled copies for later display/use.
- Standard-scales item and user features (zero mean, unit variance).
- Scales target ratings to (-1, 1) for smoother NN training.

In [ ]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y

### train/test split
- Random 80/20 split for users, items, and targets (with consistent seed).
- Prints resulting shapes.

In [ ]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

### preview normalized user training rows
- Shows normalized user rows to confirm scaling.

In [ ]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

###  build user & item “towers” (neural encoders)
- Sets embedding size (`num_outputs=32`) and random seed.
- Defines two MLP towers (user & item): 256→128→32 with ReLU except last linear layer.
- Builds Keras functional model: inputs → towers → L2-normalized embeddings → dot product output.
- Prints model summary.

In [ ]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(num_outputs, activation = 'linear')
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(num_outputs, activation = 'linear')
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

### unit tests for tower shapes
- Runs provided tests to verify the towers output the expected shapes.

In [ ]:
# Public tests
from public_tests import *
test_tower(user_NN)
test_tower(item_NN)

###  compile model
- Uses MSE loss on the scaled targets and Adam optimizer (lr=0.01).
- Fixes random seed for reproducibility.

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

###  train model
- Trains for 30 epochs on the feature slices (skipping ID/other columns).
- Learns the two towers jointly (embeddings + dense weights).

In [ ]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

###  evaluate
- Computes test MSE on held-out examples.

In [ ]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

### craft a brand-new user profile
- Creates a hypothetical user who loves Adventure and Fantasy.
- Packs the user features into a single row vector for scoring.

In [ ]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

### predict top movies for that new user
- Repeats the user vector to pair against every movie.
- Scales vectors, predicts scores, unscales predictions.
- Sorts by predicted rating and prints the top-10 titles.

In [ ]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

###  predict for an existing user (rank all movies)
- Builds all (user, movie) pairs for a given user `uid`.
- Predicts scores for every movie, sorts, and prints predictions alongside actual ratings for movies the user rated.

In [ ]:
uid = 2 
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]

#print sorted predictions for movies rated by the user
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 50)

###  implement squared distance (graded)
- Defines squared Euclidean distance (used to find nearest neighbor movies in embedding space).

In [ ]:
# GRADED_FUNCTION: sq_dist
# UNQ_C2
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    d = 0
    ### START CODE HERE ###
    for i in range(a.shape[0]):
        d += (a[i]-b[i])**2
    
    ### END CODE HERE ###     
    return d

###  quick checks for sq_dist
- Sanity tests for the distance function.

In [ ]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

###  public tests for sq_dist
- Runs grader tests to validate correctness.

In [ ]:
# Public tests
test_sq_dist(sq_dist)

###  build a model that outputs item embeddings only
- Re-wraps the trained item tower to directly output normalized item embeddings (`vm`).
- This is used to compute movie-to-movie similarity.

> Note: This cell preserves the literal `...` in the code line as previously pasted.

In [ ]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                     ...   # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

###  compute all movie embeddings
- Scales all items and passes them through the item tower to get every movie’s embedding.

In [ ]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

###  find nearest neighbor movies (by embedding distance)
- Computes pairwise squared distances between all movie embeddings.
- Masks self-distance.
- For the first `count` movies, finds the nearest neighbor by embedding distance.
- Builds an HTML table showing movie → most similar movie (and genres).

In [ ]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table